## Deliverable 3: Create a Travel Itinerary Map

In [71]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
import os
import sys

parent_folder = os.path.dirname(os.getcwd())
sys.path.append(parent_folder)

from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [72]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.
vacation_df = pd.read_csv("../Vacation_Search/WeatherPy_vacation.csv")
vacation_df.head()

,City_ID,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Boston,US,73.58,broken clouds,42.3584,-71.0598,Kimpton Nine Zero Hotel
1,New York,US,79.95,clear sky,40.7143,-74.0060,Duane Street Hotel
2,Bethesda,US,80.22,clear sky,38.9807,-77.1003,Hyatt Regency Bethesda
3,Chicago,US,82.99,clear sky,41.8500,-87.6500,Chinatown Hotel
4,Tampa,US,88.95,broken clouds,27.9475,-82.4584,Hilton Tampa Downtown


In [73]:
def get_info_box_template():
    """
    Return info box template string to be used for pop-ups on google maps
    """
    
    info_box_template = \
    """<dl>
    <dt>Hotel Name</dt><dd>{Hotel Name}</dd>
    <dt>City</dt><dd>{City_ID}</dd>
    <dt>Country</dt><dd>{Country}</dd>
    <dt>Weather Description</dt><dd>{Current Description}</dd>
    <dt>Max Temp</dt><dd>{Max Temp} °F</dd>
    </dl>
    """
    
    return info_box_template

In [76]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature for the city.
info_box_template = get_info_box_template()

# 3a. Get the data from each row and add it to the formatting template and store the data in a list.
vacation_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# 3b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_df[["Lat", "Lng"]]

In [77]:
def create_heat_map(locations, weights, marker_info=None):
    """
    create new heat map with gmaps for locations and weights passed in
    
    args:
        locations: DataFrame with two columns [["Lat", "Lng"]]
        weights: List of max temperatures or humidity, or another weather metric
    """ 
    # create figure
    fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
    
    heat_layer = gmaps.heatmap_layer(locations, weights, dissipating=False, max_intensity=300, point_radius=4)
        
    # add the heatmap layers
    fig.add_layer(heat_layer)

    # add marker layer if marker_info argument is provided
    if (marker_info != None):
        marker_layer = gmaps.marker_layer(locations, info_box_content=marker_info)
        fig.add_layer(marker_layer)

    
    # call figure to plot the data
    return fig

In [78]:
# 4a. Add a marker layer for each city to the map. 
max_temp = vacation_df["Max Temp"]
fig = create_heat_map(locations, max_temp, vacation_info)

# 4b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [79]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.

vacation_start = vacation_df.loc[vacation_df["City_ID"] == "Boston"]
vacation_end = vacation_df.loc[vacation_df["City_ID"] == "Boston"]
vacation_stop1 = vacation_df.loc[vacation_df["City_ID"] == "New York"]
vacation_stop2 = vacation_df.loc[vacation_df["City_ID"] == "Bethesda"] 
vacation_stop3 = vacation_df.loc[vacation_df["City_ID"] == "Chicago"]

In [80]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
start = vacation_start["Lat"].to_numpy()[0], vacation_start["Lng"].to_numpy()[0]
end = vacation_end["Lat"].to_numpy()[0], vacation_end["Lng"].to_numpy()[0]
stop1 = vacation_stop1["Lat"].to_numpy()[0], vacation_stop1["Lng"].to_numpy()[0]
stop2 = vacation_stop2["Lat"].to_numpy()[0], vacation_stop2["Lng"].to_numpy()[0]
stop3 = vacation_stop3["Lat"].to_numpy()[0], vacation_stop3["Lng"].to_numpy()[0]

In [81]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".
fig = gmaps.figure()

driving_itinerary = gmaps.directions_layer(start, end, waypoints=[stop1, stop2, stop3], travel_mode= 'DRIVING')

fig.add_layer(driving_itinerary)
fig

Figure(layout=FigureLayout(height='420px'))

In [82]:
# 8. To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat([vacation_start, vacation_stop1, vacation_stop2, vacation_stop3],ignore_index=True)
itinerary_df

,City_ID,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Boston,US,73.58,broken clouds,42.3584,-71.0598,Kimpton Nine Zero Hotel
1,New York,US,79.95,clear sky,40.7143,-74.0060,Duane Street Hotel
2,Bethesda,US,80.22,clear sky,38.9807,-77.1003,Hyatt Regency Bethesda
3,Chicago,US,82.99,clear sky,41.8500,-87.6500,Chinatown Hotel


In [83]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city. 
info_box_template = get_info_box_template()

# 10a Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]

In [84]:
# 11a. Add a marker layer for each city to the map.
fig = gmaps.figure()

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))